In [1]:
#local imports
from func.DataLoader import gdfLoader,ReadMarkerMap,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from func.FoliumFeatureCreation.RawDataFeatures import RawDataFeatures
from func.MapCreation import MapCreation
from Cartobat.src.API.API import API
from secret import API_KEY
from datetime import datetime
import json
import pandas as pd
from shapely.geometry import Point,Polygon,mapping
import folium
from folium.plugins import TimestampedGeoJson
import numpy as np

In [2]:
MapDataPath="data/cartePalaiseau.csv"
ModuleMapPath="data/carteModulePalaiseau.csv"
Data_Path="data/manipstatic.csv"

saving_output=False

sampling_time=2000#ms #si supérieur à 2000ms il faut implémenter la moyenne dans la liste des rssi dans marker dict list

In [3]:
#Map Data
map_gdf=gdfLoader(MapDataPath)
map_gdf.to_crs(epsg=4326, inplace=True)
#marker data
marker_gdf=ReadMarkerMap(ModuleMapPath)


In [4]:
m=MapCreation(map_gdf,marker_gdf=marker_gdf)
m.save('output/map.html')
m

In [5]:
map_gdf.head()

,ID_element,nom,geometry
0,21205,4A101,"POLYGON ((2.19977 48.71281, 2.19979 48.71287, ..."
1,21264,4A413,"POLYGON ((2.20045 48.71307, 2.20046 48.71311, ..."
2,21265,4A409,"POLYGON ((2.20043 48.71301, 2.20044 48.71304, ..."
3,21266,,"POLYGON ((2.20050 48.71315, 2.20055 48.71315, ..."
4,21267,,"POLYGON ((2.20055 48.71317, 2.20059 48.71316, ..."


In [6]:
expectation=json.loads(open('data/spf_calibration/calib1.json').read())

In [7]:
# here we get the actual data we want to predict position in zero shot
start,end="2023-06-28 17:00:00.000000","2023-06-28 17:12:00.000000"

Gatien_API=API(API_KEY)
hour_correction=2
start=pd.to_datetime(start)-pd.Timedelta(hour_correction, unit="h")
end=pd.to_datetime(end)-pd.Timedelta(hour_correction, unit="h")
data=Gatien_API.getRawDataForCartoWear("C1598FF23480",start,end)
data=pd.DataFrame(data)
data["timestamp"]=pd.to_datetime(data['timestamp'])
data


,timestamp,macModule,rssi
0,2023-06-28 15:11:59.419,A8032A311DAA,-75
1,2023-06-28 15:11:59.241,A8032A311F96,-71
2,2023-06-28 15:11:59.044,A8032A311F6A,-69
3,2023-06-28 15:11:56.349,A8032A311F6A,-64
4,2023-06-28 15:11:54.183,A8032A311DAA,-67
...,...,...,...
894,2023-06-28 15:00:12.779,A8032A311DAA,-77
895,2023-06-28 15:00:09.763,A8032A311F96,-77
896,2023-06-28 15:00:07.860,A8032A311F6A,-62
897,2023-06-28 15:00:07.549,A8032A311DAA,-77


In [8]:
#parse data to create a list of data for sampled every sampling time ms
timestamp_list= pd.date_range(start=start, end=end, freq=str(sampling_time)+"ms").tolist()
data_list=[]
for i in range(len(timestamp_list)):
    data_treated = data[(data["timestamp"] > timestamp_list[i-1]) & (data["timestamp"] <= timestamp_list[i])]
    data_treated=data_treated.groupby("macModule").mean().reset_index()
    data_treated=data_treated[["macModule","rssi"]]
    data_list.append(data_treated)


In [9]:
def PossibleRoom(data_treated,returnMacModule=False):
    room_dict={}
    for index,row in data_treated.iterrows():
        room_value=expectation[row["macModule"]]
        for room in room_value:
            if room_value[room]:
                min,max=room_value[room]
                if min<=row["rssi"]<=max:
                    if returnMacModule:room_dict[room]=room_dict.get(room,[])+[row["macModule"]]
                    else: room_dict[room]=room_dict.get(room,0)+1
    return(room_dict)


In [10]:
possible_room_list=[]
for i in data_list:
    possible_room_list.append((PossibleRoom(i,returnMacModule=True)))

In [11]:
map_gdf.head()

,ID_element,nom,geometry
0,21205,4A101,"POLYGON ((2.19977 48.71281, 2.19979 48.71287, ..."
1,21264,4A413,"POLYGON ((2.20045 48.71307, 2.20046 48.71311, ..."
2,21265,4A409,"POLYGON ((2.20043 48.71301, 2.20044 48.71304, ..."
3,21266,,"POLYGON ((2.20050 48.71315, 2.20055 48.71315, ..."
4,21267,,"POLYGON ((2.20055 48.71317, 2.20059 48.71316, ..."


In [12]:
macModule_list=marker_gdf["macModule"].to_list()


In [13]:
features = []
input_format = '%Y-%m-%d %H:%M:%S'
output_format = '%Y-%m-%dT%H:%M:%S'

    
for index,room_dict in enumerate(possible_room_list):
    temp_timestamp=timestamp_list[index]
    dt = datetime.strptime(str(temp_timestamp), input_format)
    timestamp = dt.strftime(output_format)
    for key in room_dict:
        room_list=room_dict[key]
        temp=map_gdf[map_gdf["ID_element"]==key]
        coord=temp['geometry']
        coordinates=mapping(coord)['features'][0]['geometry']['coordinates'][0]  
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': [coordinates],
            },
            'properties': {
                'times': [timestamp],
                'popup':f"{key} {room_list}",
                'style': {
                'color': 'red',
                'opacity':0.2*len(room_list),
                } }}
        features.append(feature)



In [14]:
def RawDataFeaturesGenerator(marker_gdf,raw_rssi_df,sampling_time=2000):
    MacModuleLocation = marker_gdf.set_index("macModule").to_dict()["geometry"]
    min_timestamp=min(raw_rssi_df['timestamp'])+pd.Timedelta(sampling_time, unit="ms")
    max_timestamp=max(raw_rssi_df['timestamp'])
    # set the timestamp as index
    raw_rssi_df=raw_rssi_df.sort_values('timestamp')
    #drop the duplicates
    raw_rssi_df=raw_rssi_df.drop_duplicates()
    raw_rssi_df.index = pd.RangeIndex(len(raw_rssi_df.index))
    timestamp_list= pd.date_range(start=min_timestamp, end=max_timestamp, freq=str(sampling_time)+"ms").tolist()
    MarkerDictList=getMarkerDictList(raw_rssi_df, timestamp_list)
    raw_data_features=RawDataFeatures(MarkerDictList,timestamp_list,MacModuleLocation)
    
    return raw_data_features

In [15]:
raw_data_features=RawDataFeaturesGenerator(marker_gdf,data,sampling_time=sampling_time)
    
TimedLayer=TimestampedGeoJson(
                    {"type": "FeatureCollection","features": raw_data_features+features},
                    period='PT2S',
                    duration='PT1S',
                    transition_time=sampling_time,
                    auto_play=True)
raw_data_features

[{'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [2.200638986440939, 48.71317445007082]},
  'properties': {'times': ['2023-06-28T15:00:04'],
   'icon': 'circle',
   'popup': '-77',
   'iconstyle': {'color': '#0000f7',
    'fillColor': '#0000f7',
    'fillOpacity': '0.8',
    'radius': '23'}}},
 {'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [2.200573795873236, 48.713174265469796]},
  'properties': {'times': ['2023-06-28T15:00:08'],
   'icon': 'circle',
   'popup': '-77',
   'iconstyle': {'color': '#0000f7',
    'fillColor': '#0000f7',
    'fillOpacity': '0.8',
    'radius': '23'}}},
 {'type': 'Feature',
  'geometry': {'type': 'Point',
   'coordinates': [2.200638986440939, 48.71317445007082]},
  'properties': {'times': ['2023-06-28T15:00:08'],
   'icon': 'circle',
   'popup': '-62',
   'iconstyle': {'color': '#dfff18',
    'fillColor': '#dfff18',
    'fillOpacity': '0.8',
    'radius': '38'}}},
 {'type': 'Feature',
  'geometry': {'type': 'Poi

In [16]:
m=MapCreation(map_gdf,marker_gdf=marker_gdf,location=[48.711,2.208])
TimedLayer.add_to(m)
folium.LayerControl().add_to(m)

m.save('output/spfmap.html')
m